In [6]:
import pandas as pd
import numpy as np
import sys, re, os, glob
from collections import *

In [8]:
chr_interest = 'chr22'
reference = ''.join(l.strip().upper() 
                    for l in open(chr_interest + '.fa').readlines()[1:])
len(reference)

51304566

In [9]:
df = pd.read_table("GRCh37GenomicSuperDup.tab")
df = df[(df.chrom == chr_interest) & (df.otherChrom == chr_interest) & (df.chromStart < df.otherStart)]
df['chromSize'] = df.chromEnd - df.chromStart
print(df.shape)
df = df[(df.strand == '+')].iloc[:, [0, 1, 2, 7, 8, -4, -1]]
df.head()

(258, 30)


,chrom,chromStart,chromEnd,otherStart,otherEnd,fracMatchIndel,chromSize
36882,chr22,16085070,16090861,16460104,16465880,0.967463,5791
36884,chr22,16170803,16174525,22663435,22667149,0.920356,3722
36886,chr22,16239131,16243489,16244049,16248400,0.925092,4358
36890,chr22,16472367,16478266,16878302,16884156,0.951157,5899
36896,chr22,17022038,17025489,17347878,17351328,0.965767,3451


In [11]:
df[df.otherStart==404049]

,chrom,chromStart,chromEnd,otherStart,otherEnd,fracMatchIndel,chromSize


In [10]:
hits = {}
for _, r in df.iterrows():
    hits[(r.chromStart, r.chromEnd, r.otherStart, r.otherEnd)] = list()

In [11]:
def diff(X, Y):
    sA, eA = X[:2]
    sB, eB = Y[:2]
    
    x = max(sA, sB)
    y = min(eA, eB)
    d = max(y - x, 0) 
    dA = 100.0 * d / float(eA - sA)

    sA, eA = X[2:4]
    sB, eB = Y[2:4]
    
    x = max(sA, sB)
    y = min(eA, eB)
    d = max(y - x, 0)
    dB = 100.0 * d / float(eA - sA)

    return (dA, dB)

In [30]:
with open(chr_interest + '.results.bed', 'w') as fw:
    with open('../' + chr_interest + '.results') as f:
        for i in range(6): next(f)
        for l in f:
            l = l.strip().split()
            print >>fw, '{0}\t{1}\t{2}\t{0}\t{3}\t{4}'.format(chr_interest, l[5], l[6], l[8], l[9])

In [37]:
%%bash
wc -l chr22.results.bed
cat GRCh37GenomicSuperDup.tab | sed 1d | awk '$6=="+"' | cut -f1,2,3,7,8,9 | awk '$1==$4 && $1=="chr22"' > GRCh37GenomicSuperDup.bed
bedtools pairtopair -a GRCh37GenomicSuperDup.bed -b chr22.results.bed -is -type both > chr22.intersect.bed
wc -l chr22.intersect.bed

135500 chr22.results.bed
1802 chr22.intersect.bed


In [38]:
with open(chr_interest + '.intersect.bed') as f:
    for l in f:
        l = l.strip().split()
        A = (int(l[1]), int(l[2]), int(l[4]), int(l[5]))
        B = (int(l[7]), int(l[8]), int(l[10]), int(l[11]))
        if A[0] > A[2]: continue
        if A not in hits: 
            print('>> ooops', A)
            continue
        hits[A].append((diff(A, B), B))

In [41]:
#  ((50.86688107648516, 52.56275784910146),
#   (121361171, 121418375, 121418376, 121472478),
#   (121350751, 121391239, 121444958, 121485446)),

# _ _ _ 20,711,532  20,713,302 _...          |  20,713,376  20,715,249             [2k]
# 20,708,000                      20,734,874 |  _ 20,713,972            20,740,846 [26k]

df[df.chromStart==20711532]

,chrom,chromStart,chromEnd,otherStart,otherEnd,fracMatchIndel,chromSize
37132,chr22,20711532,20713302,20713376,20715249,0.914188,1770


In [40]:
haha = []
for k, vs in hits.items():
    ok = any(v[0][0] == 100 and v[0][1] == 100 for v in vs)
    if not ok: 
        haha += sorted(
            ((min(*pf), max(*pf)), k, px) for pf, px in vs
        )
#         print(k)
#         for pf, px in vs:
#             if pf[0]<50 or pf[1]<50:
#                 print('*', end='')
#             print('    {:3.0f} {:3.0f}: {}'.format(*pf, px))
display(sorted(haha))

[((68.17939135077415, 100.0),
  (20711532, 20713302, 20713376, 20715249),
  (20708000, 20734874, 20713972, 20740846)),
 ((79.6049119060331, 100.0),
  (20711532, 20713302, 20713376, 20715249),
  (20707750, 20713573, 20709044, 20714867)),
 ((79.6049119060331, 100.0),
  (20711532, 20713302, 20713376, 20715249),
  (20707750, 20713573, 20709044, 20714867)),
 ((80.2228578238589, 100.0),
  (20609431, 20626213, 25018053, 25035107),
  (20612750, 30612750, 22986593, 32986593)),
 ((99.58884519127636, 100.0),
  (20609431, 20626213, 25018053, 25035107),
  (20609500, 30609500, 25017133, 35017133))]

In [16]:
!tail -n+7 chr1.results | cut -d' ' -f3,4,6,7,8,9 > chr1.results.fixed

In [41]:
dx = pd.read_table('chr1.results.fixed', sep=' ', 
                   names='chromStart chromEnd otherStart otherEnd fracMatchIndel preventionCause'.split())
dx.otherStart = dx.otherStart.apply(lambda x: x + 1)
dx.otherEnd = dx.otherEnd.apply(lambda x: x + 1)
dx['chromSize'] = dx.chromEnd - dx.chromStart
#dx.chromSize = dx.apply(lambda x: x['chromSize'] if x['otherEnd'] - x['otherStart'] < x['chromSize']
#                        else x['otherEnd'] - x['otherStart'], axis=1).map(int)
dx.head()

,chromStart,chromEnd,otherStart,otherEnd,fracMatchIndel,preventionCause,chromSize
0,10000,11000,0,1000,0.0000,2,1000
1,10000,12007,18579871,18581878,74.9542,2,2007
2,10000,12007,78593431,78595438,74.9542,2,2007
3,10000,11000,143273126,143274126,0.0000,2,1000
4,10000,11000,148926474,148927474,0.0000,2,1000


In [39]:
dx = dx.head()
dx['chromSize'] = dx.chromEnd - dx.chromStart
dx.chromSize = dx.apply(lambda x: x['chromSize'] if x['otherEnd'] - x['otherStart'] < x['chromSize']
                        else x['otherEnd'] - x['otherStart'], axis=1).map(int)
dx

,chromStart,chromEnd,otherStart,otherEnd,fracMatchIndel,preventionCause,chromSize
0,10000,11000,0,1000,0.0000,2,1000
1,10000,12007,18579871,18581878,74.9542,2,2007
2,10000,12007,78593431,78595438,74.9542,2,2007
3,10000,11000,143273126,143274126,0.0000,2,1000
4,10000,11000,148926474,148927474,0.0000,2,1000


In [42]:
pth = sorted((dr.chromStart, dr.chromEnd, dr.otherStart, dr.otherEnd) for d, dr in dx.iterrows())
pth[0]

KeyboardInterrupt: 

In [ ]:
from bisect import bisect_left

def binary_search(a, x, lo=0, hi=None):  # can't use a to specify default for hi
    hi = hi if hi is not None else len(a)  # hi defaults to len(a)   
    pos = bisect_left(a, x, lo, hi)  # find insertion position
    return (pos if pos != hi and a[pos] == x else -1)  # don't walk off the end


## MashMap scratch

In [ ]:
reference_mashmap = list(reference)
with open('query.fa', 'w') as f:
    for ri, r in list(df.iterrows())[:5]:
        print >>f, '>seq{} {}-{} {}-{} ({} or {})'.format(
           ri, r.chromStart, r.chromEnd, r.otherStart, r.otherEnd, r.fracMatch, r.fracMatchIndel)
        print >>f, reference[r.chromStart:r.chromEnd]
        for i in xrange(r.chromStart, r.chromEnd):
            reference_mashmap[i] = 'N'
with open('reference.fa', 'w') as f:
    print >>f, '>chr1'
    print >>f, ''.join(reference_mashmap)

In [4]:
!time ./MashMap/mashmap -s reference.fa -q query.fa -o query.mashmap -m 1000

>>>>>>>>>>>>>>>>>>
Reference = [reference.fa]
Query = [query.fa]
Kmer size = 16
Window size = 22
Minimum alignment length >= 1000
Alphabet = DNA
Percentage identity threshold = 85
Mapping output file = query.mashmap
Filter mode = 1 (1 = map, 2 = one-to-one, 3 = none)
>>>>>>>>>>>>>>>>>>
INFO, skch::Sketch::build, minimizers picked from reference = 19596618
INFO, skch::Sketch::index, unique minimizers = 13676516
INFO, skch::Sketch::computeFreqHist, Frequency histogram of minimizers = (1, 11618003) ... (23956, 1)
INFO, skch::Sketch::computeFreqHist, With threshold 0.001%, ignore minimizers occurring >= 1572 times during lookup.
INFO, skch::main, Time spent computing the reference index: 20.7821 sec
INFO, skch::Map::mapQuery, [count of mapped reads, reads qualified for mapping, total input reads] = [5, 5, 5]
INFO, skch::main, Time spent mapping the query : 0.0854692 sec
INFO, skch::main, mapping results saved in : query.mashmap
27.51user 0.74system 0:28.34elapsed 99%CPU (0avgtext+0avgdata 

In [10]:
dm = pd.read_table("query.mashmap", sep=' ', names='rname len start end strand chrName chrLen chrStart chrEnd sim j w'.split())
dm[dm.strand == '+']

,rname,len,start,end,strand,chrName,chrLen,chrStart,chrEnd,sim,j,w
1,seq4,1136,0,1135,+,chr1,249250621,521351,522486,97.4217,47,95
4,seq40,1134,0,1133,+,chr1,249250621,521366,522499,97.7159,51,96


In [5]:
!cat chr1.s2 | grep -v '^#' | cut -f2-8 > chr1.s2_coord
!head -n2 chr1.s2_coord

chr1	137324	137573	0	chr1	327153	327402
chr1	135051	135300	0	chr1	138045	138294


In [6]:
df = pd.read_table("chr1.s2_coord", 
                   names='chrom chromStart chromEnd strand otherChrom otherStart otherEnd'.split())
df = df[(df.chrom == "chr1") & (df.otherChrom == "chr1") & (df.chromStart < df.otherStart)]
df['chromSize'] = df.chromEnd - df.chromStart
df['otherSize'] = df.otherEnd - df.otherStart
print df.shape
df[(df.otherSize < 3000) & (df.strand < 1)].head()

(17727, 9)


,chrom,chromStart,chromEnd,strand,otherChrom,otherStart,otherEnd,chromSize,otherSize
0,chr1,137324,137573,0,chr1,327153,327402,249,249
1,chr1,135051,135300,0,chr1,138045,138294,249,249
3,chr1,135559,135808,0,chr1,137662,137911,249,249
8,chr1,137326,137607,0,chr1,228159632,228159913,281,281
9,chr1,137326,137607,0,chr1,228160763,228161044,281,281
